In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
import warnings
import joblib

warnings.filterwarnings('ignore')

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
from src.model import train, test
from src.model.model import (
    LightGBMModel,
    XGBoostModel,
    CatBoostModel,
    LogisticRegressionModel,
    StackingEnsemble
)
from src.utils import get_config

In [4]:
config = get_config.read_yaml_from_main()
print("Configuration loaded successfully!")

Configuration loaded successfully!


In [5]:
def run_model(model_name):
    model_dir = Path(config['paths']['model_data_directory'])
    model_path = model_dir / f"{model_name}_model.joblib"
    train.train_model(model_name, model_path)
    test.test_model(model_name)

In [6]:
model_name = 'lightgbm'
run_model(model_name)

--- Preparing to Train Model: lightgbm ---
--- Fitting LightGBMModel ---
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 17377, number of negative: 197880
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.064601 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15686
[LightGBM] [Info] Number of data points in the train set: 215257, number of used features: 96
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Validation ROC AUC: 0.6125
Validation PR AUC (AUC-PR): 0.1277
Model saved to ..\models\lightgbm_model.joblib
--- Testing Model from: ../models/lightgbm_model.joblib ---

--- Test Set Performance ---
Test ROC AUC: 0.6133
Test PR AUC: 0.1268


In [7]:
model_name = 'xgboost'
run_model(model_name)

--- Preparing to Train Model: xgboost ---
--- Fitting XGBoostModel ---
Validation ROC AUC: 0.5797
Validation PR AUC (AUC-PR): 0.1156
Model saved to ..\models\xgboost_model.joblib
--- Testing Model from: ../models/xgboost_model.joblib ---

--- Test Set Performance ---
Test ROC AUC: 0.5875
Test PR AUC: 0.1158


In [8]:
# model_name = 'catboost'
# run_model(model_name)

In [9]:
model_name = 'logistic_regression'
run_model(model_name)

--- Preparing to Train Model: logistic_regression ---
--- Fitting LogisticRegressionModel ---
Validation ROC AUC: 0.6114
Validation PR AUC (AUC-PR): 0.1242
Model saved to ..\models\logistic_regression_model.joblib
--- Testing Model from: ../models/logistic_regression_model.joblib ---

--- Test Set Performance ---
Test ROC AUC: 0.6140
Test PR AUC: 0.1263


In [10]:
# model_name = 'ensemble'
# run_model(model_name)

In [ ]:
import optuna
from sklearn.metrics import roc_auc_score

# --- This code goes in a new cell in your notebook ---

def objective(trial):

    data_dir = config['paths']['processed_data_directory']
    train_df = pd.read_csv(data_dir + 'clean_train_data.csv')
    val_df = pd.read_csv(data_dir + "/clean_val_data.csv")

    # 2. Prepare Data
    id_col = config['data']['id']
    target_col = config['data']['target']

    X_train = train_df.drop(columns=[id_col, target_col])
    y_train = train_df[target_col]
    X_val = val_df.drop(columns=[id_col, target_col])
    y_val = val_df[target_col]

    # 1. Define the hyperparameters to search
    params = {
        'objective': 'binary',
        'metric': 'aucpr', # Optimize directly for PR AUC
        'n_estimators': trial.suggest_int('n_estimators', 200, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 200),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'class_weight': 'balanced',
        'random_state': 42,
        'verbose': -1
    }

    # 2. Create Base Model Instances with the Tuned Parameters

    # Using the model classes from your src/model/model.py
    model = LightGBMModel(params)
    model.fit(X_train, y_train)

    val_preds = model.predict_proba(X_val)[:, 1]
    roc_auc = roc_auc_score(y_val, val_preds)

    return roc_auc

# --- Create and run the study ---
# We want to MAXIMIZE the PR AUC, so the direction is 'maximize'
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=250) # Run 50 trials

print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value:.4f}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# You can now retrain your model using these best parameters.

[I 2025-08-31 01:24:23,057] A new study created in memory with name: no-name-a4c56d20-1f71-407a-bdb4-5da49a2d2bd9


--- Fitting LightGBMModel ---


[I 2025-08-31 01:24:33,027] Trial 0 finished with value: 0.6101183853168081 and parameters: {'n_estimators': 912, 'learning_rate': 0.026546647363322858, 'num_leaves': 193, 'max_depth': 5, 'min_child_samples': 124, 'feature_fraction': 0.8591028789629291, 'bagging_fraction': 0.9528020111522623, 'bagging_freq': 7, 'lambda_l1': 4.6629043400251597e-07, 'lambda_l2': 2.6666708478138462e-06}. Best is trial 0 with value: 0.6101183853168081.


--- Fitting LightGBMModel ---


[I 2025-08-31 01:24:39,844] Trial 1 finished with value: 0.5832335394261694 and parameters: {'n_estimators': 505, 'learning_rate': 0.1371697410246986, 'num_leaves': 291, 'max_depth': 5, 'min_child_samples': 117, 'feature_fraction': 0.7491070729141187, 'bagging_fraction': 0.9626997942385456, 'bagging_freq': 1, 'lambda_l1': 0.006413301450857833, 'lambda_l2': 4.867846006457378e-07}. Best is trial 0 with value: 0.6101183853168081.


...

--- Fitting LightGBMModel ---


[I 2025-08-31 02:01:45,594] Trial 156 finished with value: 0.6188552653680316 and parameters: {'n_estimators': 1012, 'learning_rate': 0.02383303058816689, 'num_leaves': 223, 'max_depth': 3, 'min_child_samples': 31, 'feature_fraction': 0.7622118555376459, 'bagging_fraction': 0.8341372671399663, 'bagging_freq': 3, 'lambda_l1': 0.0004130646110363377, 'lambda_l2': 0.026844898010403084}. Best is trial 154 with value: 0.6219416486881425.


--- Fitting LightGBMModel ---


[I 2025-08-31 02:01:53,899] Trial 157 finished with value: 0.6208299371741046 and parameters: {'n_estimators': 866, 'learning_rate': 0.015102116945431782, 'num_leaves': 243, 'max_depth': 3, 'min_child_samples': 27, 'feature_fraction': 0.7208357640928507, 'bagging_fraction': 0.8512058311938918, 'bagging_freq': 3, 'lambda_l1': 0.0009901841733277658, 'lambda_l2': 0.007465583089660178}. Best is trial 154 with value: 0.6219416486881425.


--- Fitting LightGBMModel ---
